In [1]:
import pandas as pd
import numpy as np
import re
import json

In [137]:
with open("math23k_dolphin.json") as f:
    data = json.load(f)
df = pd.read_json(data)

In [138]:
df.shape

(8, 32532)

In [139]:
answers = df.iloc[4].to_numpy()
print(answers)

['20' '1466' '180' ... '77.7241' '100.2628' '9.6955']


In [140]:
ids = np.array(df.columns).astype('str')

In [141]:
len(ids), ids

(32532, array(['0', '1', '2', ..., '32529', '32530', '32531'], dtype='<U21'))

In [142]:
ans_dict = {}
for cur_id, cur_ans in zip(ids, answers):
    ans_dict[cur_id] = cur_ans

print(ans_dict)

{'0': '20', '1': '1466', '2': '180', '3': '32', '4': '39.76', '5': '4', '6': '10', '7': '14', '8': '105', '9': '2304', '10': '12.5', '11': '39', '12': '40', '13': '158', '14': '11', '15': '85', '16': '8000', '17': '65', '18': '6.93', '19': '43', '20': '6', '21': '15', '22': '117', '23': '100', '24': '650', '25': '0.72', '26': '1800', '27': '2', '28': '179.82', '29': '10', '30': '17.5', '31': '0.5714285714285714', '32': '300', '33': '1.8', '34': '880', '35': '4', '36': '2090', '37': '300', '38': '4', '39': '93', '40': '2000', '41': '1.25', '42': '60', '43': '9.7', '44': '1664', '45': '4', '46': '160', '47': '480', '48': '60', '49': '144', '50': '2006', '51': '3.25', '52': '15972', '53': '475', '54': '73', '55': '91', '56': '0.4666666666666667', '57': '8', '58': '9', '59': '6', '60': '2340', '61': '634.5', '62': '471', '63': '77', '64': '14.4', '65': '20', '66': '1172.2', '67': '20', '68': '34', '69': '60', '70': '36', '71': '39', '72': '200', '73': '45', '74': '0.12121212121212122', '75

In [143]:
P40 = int(0.4 * len(ids))
print(P40)

13012


In [144]:
import os
def write_json(fname, data):
    with open(fname, 'w') as fp:
        json.dump(data, fp)

In [145]:
valid_ids = np.random.choice(ids, P40)
test_size = int(P40/2)
valid_size = P40 - test_size
write_json('valid_ids.json', valid_ids[:valid_size].tolist())


In [159]:
b=set(valid_ids)
len(b),len(valid_ids)
len(valid_ids)-len(b)
len(set(ids))

32532

In [146]:
# Write test data

fp1 = open('id_ans_test', 'w')

for id in valid_ids[valid_size:]:
    print(id + '\t' + ans_dict[id], file=fp1)

fp1.close()

In [147]:
eqs = df.iloc[1].to_numpy()

In [148]:
eqs

array([list(['x', '=', '(', 'temp_b', '-', '1', ')', '*', 'temp_a']),
       list(['x', '=', 'temp_b', '+', 'temp_d', '*', '(', 'temp_e', '-', 'temp_a', ')']),
       list(['x', '=', '(', 'temp_c', '+', 'temp_c', '*', 'temp_e', ')', '/', '(', '1', '-', 'temp_a', '-', 'temp_a', '*', 'temp_e', '-', 'temp_f', ')']),
       ...,
       list(['x', '=', 'temp_a', '+', 'temp_b', '+', 'temp_c', '+', 'temp_d', '+', 'temp_e']),
       list(['x', '=', 'temp_a', '+', 'temp_b', '+', 'temp_c', '+', 'temp_d', '+', 'temp_e']),
       list(['x', '=', 'temp_a', '+', 'temp_b', '+', 'temp_c', '+', 'temp_d', '+', 'temp_e'])],
      dtype=object)

In [149]:
#  Convert infix expression to suffix expression
def postfix_equation(equ_list):
    try:
        stack = []
        post_equ = []
        op_list = ['+', '-', '*', '/', '^']
        priori = {'^': 3, '*': 2, '/': 2, '+': 1, '-': 1}
        for elem in equ_list:
            if elem == '(':
                stack.append('(')
            elif elem == ')':
                while 1:
                    op = stack.pop()
                    if op == '(':
                        break
                    else:
                        post_equ.append(op)
            elif elem in op_list:
                while 1:
                    if not stack:
                        break
                    elif stack[-1] == '(':
                        break
                    elif priori[elem] > priori[stack[-1]]:
                        break
                    else:
                        op = stack.pop()
                        post_equ.append(op)
                stack.append(elem)
            else:
                post_equ.append(elem)
        while stack:
            post_equ.append(stack.pop())
        return post_equ
    except:
        print(equ_list)

In [150]:
post_list = []
for eq in eqs:
    post_list.append(postfix_equation(eq[2:]))

In [151]:
post_list

[['temp_b', '1', '-', 'temp_a', '*'],
 ['temp_b', 'temp_d', 'temp_e', 'temp_a', '-', '*', '+'],
 ['temp_c',
  'temp_c',
  'temp_e',
  '*',
  '+',
  '1',
  'temp_a',
  '-',
  'temp_a',
  'temp_e',
  '*',
  '-',
  'temp_f',
  '-',
  '/'],
 ['temp_a', 'temp_b', '*', 'temp_c', '/'],
 ['temp_a', 'temp_b', '*'],
 ['temp_a', 'temp_b', 'temp_c', '*', '-', 'temp_d', '/'],
 ['temp_e', 'temp_c', '-', 'temp_a', '/'],
 ['temp_a', 'temp_b', '/'],
 ['temp_a', 'temp_b', '*', 'temp_c', '/'],
 ['temp_d', 'temp_b', '/', 'temp_c', '/'],
 ['temp_a', 'temp_a', 'temp_b', '-', 'temp_d', '*', '-', 'temp_b', '-'],
 ['temp_a', 'temp_a', 'temp_b', '/', 'temp_c', '+', '/'],
 ['temp_a', 'temp_d', '-', 'temp_b', 'temp_c', '*', '-', 'temp_c', '/'],
 ['temp_a', 'temp_b', '*', 'temp_c', 'temp_d', '*', '-', 'temp_e', '/'],
 ['temp_a', 'temp_b', '-'],
 ['temp_a', 'temp_b', '*', 'temp_c', '+'],
 ['temp_a', 'temp_b', '*', 'temp_c', '-'],
 ['temp_e', 'temp_d', '+', 'temp_a', 'temp_b', '-', 'temp_b', '-', '/'],
 ['temp_b', '

In [152]:
post_dict = {}
for cur_id, cur_post in zip(ids, post_list):
    post_dict[cur_id] = cur_post

print(post_dict)

{'0': ['temp_b', '1', '-', 'temp_a', '*'], '1': ['temp_b', 'temp_d', 'temp_e', 'temp_a', '-', '*', '+'], '2': ['temp_c', 'temp_c', 'temp_e', '*', '+', '1', 'temp_a', '-', 'temp_a', 'temp_e', '*', '-', 'temp_f', '-', '/'], '3': ['temp_a', 'temp_b', '*', 'temp_c', '/'], '4': ['temp_a', 'temp_b', '*'], '5': ['temp_a', 'temp_b', 'temp_c', '*', '-', 'temp_d', '/'], '6': ['temp_e', 'temp_c', '-', 'temp_a', '/'], '7': ['temp_a', 'temp_b', '/'], '8': ['temp_a', 'temp_b', '*', 'temp_c', '/'], '9': ['temp_d', 'temp_b', '/', 'temp_c', '/'], '10': ['temp_a', 'temp_a', 'temp_b', '-', 'temp_d', '*', '-', 'temp_b', '-'], '11': ['temp_a', 'temp_a', 'temp_b', '/', 'temp_c', '+', '/'], '12': ['temp_a', 'temp_d', '-', 'temp_b', 'temp_c', '*', '-', 'temp_c', '/'], '13': ['temp_a', 'temp_b', '*', 'temp_c', 'temp_d', '*', '-', 'temp_e', '/'], '14': ['temp_a', 'temp_b', '-'], '15': ['temp_a', 'temp_b', '*', 'temp_c', '+'], '16': ['temp_a', 'temp_b', '*', 'temp_c', '-'], '17': ['temp_e', 'temp_d', '+', 'temp_

In [153]:
write_json('op_post_dup_templates.json', post_dict)